# SweepRunner Tutorial

This tutorial demonstrates how to use the `SweepRunner` class from `src.utils` to automate parameter sweeps for experiments. The `SweepRunner` handles logging, parallelization, reproducibility, and allows you to pause and resume sweeps at any time.

## Key Features
- **Automated logging** of all results and parameters
- **Parallel execution** of trials
- **Reproducibility** via controlled random seeds
- **Comparibility** by allowing synchronised seeds for specific parameters, e.g. methods
- **Pause and resume**: interrupted sweeps can be continued
- **Flexible API**: just provide your trial function, a sweep space, and (optionally) a filter for valid sweeps


### 1. Imports

To access the files in `src/` from within this notebook, we need to add the root directory to the Python path.

In [ ]:
import numpy as np
import sys
import os
# Get the absolute root path of this repository
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
# Add it to the Python path
if root_path not in sys.path:
    sys.path.insert(0, root_path)

from src.gamecore import SweepRunner

## 2. Define your trial function

Your function should have the signature:
```python
def run_trial_fn(seed: int, sweep_params: dict, **kwargs) -> str:
    ...
    return outcome
```
It should run a single experiment/trial and return a result (e.g., a string label or a metric).

In [3]:
def run_trial_fn(seed: int, sweep_params: dict, **kwargs) -> str:
    
    def random_walk_outcome(steps: int, threshold: float) -> str:
        """ Simulate a random walk and determine if it diverges or remains stable."""
        walk = np.cumsum(np.random.randn(steps))
        if np.max(np.abs(walk)) > threshold:
            return "diverged"
        else:
            return "stable"
        
    np.random.seed(seed)
    # Example: simple random walk outcome
    method = sweep_params["method"]
    steps = sweep_params["steps"]
    threshold = sweep_params["threshold"]
    if method == "A":
        # Method A: Random walk with a threshold
        return random_walk_outcome(steps, threshold)
    elif method == "B":
        # Method B: Different random walk logic
        return random_walk_outcome(steps, threshold * 2)  # Just an example of different logic
    else:
        raise ValueError(f"Unknown method: {method}")

## 3. (Optional) Define a sweep filter

You can provide a function to skip invalid or uninteresting parameter combinations.

In [4]:
def is_valid_sweep_fn(sweep_params: dict) -> bool:
    # Example: exclude sweeps with low thresholds and high steps
    if sweep_params["threshold"] < 1 and sweep_params["steps"] >= 20:
        return False
    return True

## 4. Define your sweep space

This is a dictionary mapping parameter names to lists of values. The `SweepRunner` will cover all possible combinations which are not excluded by your `is_valid_sweep_fn`.

In [5]:
sweep_space = {
    "method": ["A", "B"],
    "steps": [10, 15, 20, 25],
    "threshold": [0.5, 2, 4],
}

## 5. Run the sweep

Now simply create a `SweepRunner` and call `.run()` (optionally with additional kwargs for your `run_trial_fn`).

- All results and logs will be saved in `data/<experiment_name>`.
- You can interrupt and resume the sweep at any time.
- All seeds and parameters are managed automatically.
- If you, e.g., want to compare different methods, you can ensure comparibility by synchronising the seeds with `seed_sync_by`.

In [6]:
sweeper = SweepRunner(
    experiment_name="tutorial_sweep_runner",
    base_dir=os.path.join(root_path, "data"), # necessary, as we are inside a Jupyter notebook
    sweep_space=sweep_space,
    seed_sync_by=["method"],  # ensure comparibility by synchronising seeds across methods
    is_valid_sweep_fn=is_valid_sweep_fn,  # optional
    run_trial_fn=run_trial_fn,
    n_trials=5,  # number of repetitions per sweep
    parallel=False, # parallel does not work in Jupyter notebooks :(
)

# Run the sweep (can be interrupted/resumed)
sweeper.run()

📁 Creating new experiment directory: /home/felix/gamecore/data/tutorial_sweep_runner

All parameter combinations: 24 (potentially contain invalid combinations)
Valid parameter combinations: 20

Running sequentially ...



100%|██████████| 20/20 [00:00<00:00, 304.53it/s]



=== Running sweep 1/20: sweep_001 ===
Parameters: {'method': 'A', 'steps': 10, 'threshold': 0.5}
Sweep 001/020 - Trial 005/005 - Status: diverged=5
=== Results for sweep_001 ===
diverged: 5
Mean trial time: 0.114ms ± 0.153ms

Current progress:


=== Running sweep 2/20: sweep_002 ===
Parameters: {'method': 'A', 'steps': 10, 'threshold': 2}
Sweep 002/020 - Trial 005/005 - Status: diverged=4, stable=1
=== Results for sweep_002 ===
diverged: 4
stable: 1
Mean trial time: 0.020ms ± 0.017ms

Current progress:


=== Running sweep 3/20: sweep_003 ===
Parameters: {'method': 'A', 'steps': 10, 'threshold': 4}
Sweep 003/020 - Trial 005/005 - Status: stable=4, diverged=1
=== Results for sweep_003 ===
stable: 4
diverged: 1
Mean trial time: 0.038ms ± 0.027ms

Current progress:


=== Running sweep 4/20: sweep_004 ===
Parameters: {'method': 'A', 'steps': 15, 'threshold': 0.5}
Sweep 004/020 - Trial 005/005 - Status: diverged=5
=== Results for sweep_004 ===
diverged: 5
Mean trial time: 0.055ms ± 0.045ms

## 6. Analyze your results

All results are stored in `data/tutorial_sweeprunner/` and can be loaded with, e.g., pandas for further analysis.

See also the [analysis notebook](../analyse_sweep.ipynb) for a ready-made analysis!